In [7]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd

from scipy import stats

from dependencies import (
    perform_kmeans_clustering, 
    get_annual_return, 
    perform_polynomial_on_clustering, 
    combine_polynomial_plots,
    POLYNOMIAL_DEGREE
)

In [8]:
data_frame = pd.read_csv('datasets/S&P500_EOD-Price.csv')
data_frame = data_frame.dropna(axis=1)

data_frame.head()

,Date,APA,MTCH,GEN,DXC,EG,BKR,AMD,HON,BALL,...,SNPS,SWKS,ZION,TTWO,VRTX,TRMB,ZBRA,LNT,BBWI,WBA
0,2000-01-03,14.772727,7.527071,7.109375,46.327873,21.8750,20.2500,15.500,54.044136,2.488281,...,32.781250,31.406250,55.5000,9.125000,18.78125,4.145833,25.027779,13.40625,16.395514,28.5625
1,2000-01-04,14.123377,6.921444,6.781250,46.017365,21.8125,20.2500,14.625,53.090767,2.406250,...,30.437500,29.562500,52.8125,8.875000,17.28125,4.072917,24.666668,13.40625,16.067099,27.5000
2,2000-01-05,14.529221,6.661890,6.375000,42.788074,23.2500,20.3750,15.000,52.375736,2.449219,...,30.703125,30.328125,52.7500,8.833333,17.00000,3.817708,25.138889,14.06250,15.839733,27.8125
3,2000-01-06,15.909091,6.938748,6.007813,43.409092,24.3125,22.3125,16.000,53.388695,2.488281,...,28.468750,27.375000,53.5000,8.750000,16.75000,3.854167,23.777779,14.06250,15.587106,27.0000
4,2000-01-07,15.854978,6.904141,6.484375,43.315941,25.6250,23.8125,16.250,56.248802,2.417969,...,28.750000,30.187500,53.6250,9.000000,18.21875,3.854167,23.513889,14.25000,15.688157,27.6875


In [9]:
data_frame["Date"] = pd.to_datetime(data_frame["Date"])
data_frame["Year"] = data_frame["Date"].dt.year
data_frame["Month"] = data_frame["Date"].dt.month

processing_years = [year for year in range(2000, 2024)]
months = [month for month in range(1, 13)]
data_frame_monthly_returns = []
 
for year in processing_years:
    for month in months:
        filtered_data = data_frame[(data_frame['Year'] == year) & (data_frame['Month'] == month)]
        for column in filtered_data.columns:
            if column not in ["Date", "Year", "Month"] and not filtered_data.empty:
                monthly_return = get_annual_return(filtered_data[column].iloc[0], filtered_data[column].iloc[-1])
                monthly_returns = [get_annual_return(filtered_data[column].iloc[0], filtered_data[column].iloc[i]) for i in range(len(filtered_data))]  
               
                relative_percentile = stats.percentileofscore(monthly_returns, monthly_return)
                data_frame_monthly_returns.append({'Stock': column, 'Year': year, 'Month': month, 'Monthly_Return': monthly_return, 'Relative_Percentile': relative_percentile})

data_frame_monthly_percentile = pd.DataFrame(data_frame_monthly_returns)
data_frame_monthly_percentile.to_csv('datasets/monthly_data_return.csv', index=False)  